# Merge Bike Counter Data with 2024 Speed Limits and Trim for ease-of-access

In this notebook we will integrate data from the dataset obtained as the result of running [rdf-to-csv.ipynb](rdf-to-csv.ipynb) with another dataset about speed limits in Bologna (accessible [here](https://opendata.comune.bologna.it/explore/dataset/velocita-citta-30/)).

To do so we will:

1. Load and filter bike counter readings from January 16th 2024 onward
2. Load the 2024 speed‐limit dataset, where streets are represented as LineString/MultiLineString
3. Perform a spatial “nearest” join to associate each bike counter point with the speed limit of the closest street segment
4. Export the merged result to a new CSV file
5. Trim the dataset to daily measurements

In [13]:
import json
from pathlib import Path

import pandas as pd            # tabular data handling
import geopandas as gpd        # spatial data handling
from shapely.geometry import Point, shape  # point & geojson→geometry

## File paths

Modify these paths as needed to point to your downloaded files.

In [14]:
# Path to the CSV of bike counters
bike_csv = Path("colonnine-conta-bici.csv")

# Path to the speed limits CSV
speed_csv = Path("velocita-citta-30.csv")

# Output path for the merged CSV
merged_csv = Path("bike-merged.csv")

#Output path for the trimmed csv
trimmed_csv = Path("bike-trimmed.csv")

## Define loader functions

Define two helper functions to read into GeoDataFrames:
- `load_bike_counters` reads all readings (no date filter) and parses points.
- `load_speed_limits` reads street lines (GeoJSON) and builds geometries.

In [15]:
def load_bike_counters(path: Path) -> gpd.GeoDataFrame:
    df = pd.read_csv(path, parse_dates=['data'])
    def to_point(s):
        if pd.isna(s): return None
        lat, lon = [x.strip() for x in s.split(',')]
        return Point(float(lon), float(lat))
    return gpd.GeoDataFrame(df,
                            geometry=df['geo_point_2d'].apply(to_point),
                            crs="EPSG:4326")

def load_speed_limits(path: Path) -> gpd.GeoDataFrame:
    df = pd.read_csv(path, sep=';', encoding='utf-8-sig')
    if 'Geo Shape' not in df:
        raise ValueError("Missing 'Geo Shape' column")
    def parse_shape(js):
        if pd.isna(js): return None
        return shape(json.loads(js))
    return gpd.GeoDataFrame(df,
                            geometry=df['Geo Shape'].apply(parse_shape),
                            crs="EPSG:4326")

## Mapping
Extract one representative point per column, perform a single nearest-neighbor join against street lines (in a metric CRS), and build a mapping DataFrame.

In [16]:
# 1) Load full bike data
bike_full = load_bike_counters(bike_csv)

# 2) Unique point per column
unique_pts = (bike_full
              .drop_duplicates(subset="colonnina")
              .loc[:, ["colonnina", "geometry"]]
              .to_crs(epsg=32632))

# 3) Load and reproject street lines
speed_gdf = load_speed_limits(speed_csv).to_crs(epsg=32632)

# 4) Spatial join once to get VEL2024 per colonnina
mapping = (gpd.sjoin_nearest(unique_pts,
                             speed_gdf[['VEL2024','geometry']],
                             how='left',
                             distance_col='dist_m')
           .drop(columns=['geometry','dist_m','index_right'])
           .set_index('colonnina')['VEL2024']
           .rename('VEL2024')
           .reset_index())

## Merge function

Merge the `VEL2024` mapping back onto all readings via pandas merge, drop geometry, and write out the final CSV.

In [17]:
# Ensure full dataset in lat/lon (merge ignores geometry)
bike_full = bike_full.to_crs(epsg=4326)

# Merge
merged = bike_full.merge(mapping, on="colonnina", how="left")

# Export (dropping geometry)
merged.drop(columns='geometry').to_csv(merged_csv, index=False)
print(f"[OK] {len(merged)} rows written to {merged_csv}")

[OK] 298436 rows written to bike-merged.csv


## Exemplar Visualization
Load the merged CSV and display the first 10 rows with all columns.


In [18]:
df = pd.read_csv(merged_csv)
pd.set_option('display.max_columns', None)
display(df.head(10))

,colonnina,totale,direzione_periferia,direzione_centro,geo_point_2d,data,VEL2024
0,Orti_II,3.0,3.0,NaN,"44.47624162079627,11.37609044129954",2025-06-09 21:00:00+00:00,50
1,Murri_I,5.0,5.0,NaN,"44.48440745989066,11.35658715560829",2025-06-09 20:00:00+00:00,50
2,Mazzini_II,130.0,NaN,130.0,"44.48936290709702,11.35940582976347",2025-06-10 05:00:00+00:00,30
3,Sturzo_II,14.0,14.0,NaN,"44.48820778081575,11.29599058158018",2025-06-10 07:00:00+00:00,50
4,Massarenti_II,33.0,NaN,33.0,"44.49300884795814,11.37056742338153",2025-06-10 02:00:00+00:00,30
5,Zanardi_I,NaN,NaN,NaN,"44.50817288812807,11.32990298071598",2025-06-09 22:00:00+00:00,30
6,Murri_II,7.0,NaN,7.0,"44.48418981668801,11.35719458905058",2025-06-09 21:00:00+00:00,30
7,Zanardi_II,191.0,191.0,NaN,"44.5082267800901,11.32964907760978",2025-06-10 04:00:00+00:00,30
8,Zanardi_II,1.0,1.0,NaN,"44.5082267800901,11.32964907760978",2025-06-09 20:00:00+00:00,30
9,Zanardi_II,68.0,68.0,NaN,"44.5082267800901,11.32964907760978",2025-06-10 08:00:00+00:00,30


## Trim-to-daily function

In order to ease future processing of data, we run this Python script that:

1. **Reads** the raw hourly CSV (`bike-merged.csv`).  
2. **Parses** the timestamp column into a simple date for grouping.  
3. **Normalizes** counter names by stripping any `_I`/`_II` suffix so that split‐direction sensors merge into one.  
4. **Fills** missing hourly counts with zero to avoid gaps.  
5. **Groups** by (`colonnina`, `data`) and:
   - **Sums** all hourly counts into a daily total.  
   - **Carries over** the first observed `VEL2024` value for that counter.  
6. **Writes** the resulting daily summary to `bike-trimmed.csv`.

This will produce a clean dataset trimmed to daily measurements (instead of hourly).


In [19]:
import pandas as pd

# read input file
df = pd.read_csv(merged_csv)

# parse date from timestamp
df['data'] = pd.to_datetime(df['data']).dt.date

# strip suffix _I or _II to unify counters
df['colonnina'] = df['colonnina'].str.replace(r'(_I|_II)$', '', regex=True)

# fill missing counts
df['totale'] = df['totale'].fillna(0)

# sum hourly counts per day and counter, keep first VEL2024
daily = (
    df.groupby(['colonnina', 'data'], as_index=False)
      .agg(totale=('totale', 'sum'), VEL2024=('VEL2024', 'first'))
)

# write daily summary to CSV
daily.to_csv(trimmed_csv, index=False)

print(f"bike-trimmed.csv created with {len(daily)} records.")


bike-trimmed.csv created with 10718 records.


## Exemplar Visualization
Load the trimmed CSV and display the first 10 rows with all columns.


In [20]:
df = pd.read_csv(trimmed_csv)
pd.set_option('display.max_columns', None)
display(df.head(10))

,colonnina,data,totale,VEL2024
0,Ercolani,2018-12-31,19.0,50
1,Ercolani,2019-01-01,464.0,50
2,Ercolani,2019-01-02,816.0,50
3,Ercolani,2019-01-03,987.0,50
4,Ercolani,2019-01-04,1037.0,50
5,Ercolani,2019-01-05,704.0,50
6,Ercolani,2019-01-06,713.0,50
7,Ercolani,2019-01-07,1747.0,50
8,Ercolani,2019-01-08,1884.0,50
9,Ercolani,2019-01-09,1985.0,50
